# What is This?

Refer to the problem on OpenAIs [Site](https://gym.openai.com/envs/Taxi-v2/)

We are the taxi. Its yellow when free and green when on a journey with a passenger. The pickup is done at the Blue marker and Drop at the Pink one. 

In [1]:
import gym

env = gym.make("Taxi-v2").env

env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# Spaces

The action space is the set of actions it can perform at a particular situation.
The State Space is the number of such situations possbible in the game.

In [2]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


# Call for a State

We can create our situation with the encode functions.

#### Encode Args = taxi row, taxi column, passenger index, destination index

In [3]:
state = env.encode(3, 1, 1, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 324
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# Summary

We can view the summrary of a loaction with the given below method. A dictionary is returned with the 

#### {action: [(probability, nextstate, reward, done)]}


In [4]:
env.P[324]

{0: [(1.0, 424, -1, False)],
 1: [(1.0, 224, -1, False)],
 2: [(1.0, 344, -1, False)],
 3: [(1.0, 324, -1, False)],
 4: [(1.0, 324, -10, False)],
 5: [(1.0, 324, -10, False)]}

### Interpretation
A few things to note:

* The 0-5 corresponds to the actions (south, north, east, west, pickup, dropoff) the taxi can perform at our current state in the illustration.

* In this env, probability is always 1.0.

* The nextstate is the state we would be in if we take the action at this index of the dict

* All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)

* done is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode

# BruteForce Solving

We can use an algorithm to infinitly do random steps until the goal is reached. Its very straightfoward but ineffective and crude.

In [5]:
env.s = 328 # <<<--- Use a the state #328 to train

epochs, penalties, rewards = 0, 0, 0
# INIT all variables

frames = [] # <<<--- To store the frames to view the progress later

done = False # <<<--- Flag for checking the exit condition / goal reached

while not done:
    action = env.action_space.sample()   # <<<---Method Automatically selects one random action from set of possible actions (0-5)
    state, rewards, done, info = env.step(action) # <<<--- Performs a step with the action passed on the environment
    
    if rewards == -10:   # <<<--- Something Wrong
        penalties += 1
        
    frames.append({
        'frame'   : env.render(mode = 'ansi'),
        'state'   : state,
        'action'  : action,
        'rewards' : rewards
    })
    
    epochs += 1
print("Epochs taken =        ", epochs)
print("Number of penalties = ", penalties)

Epochs taken =         687
Number of penalties =  221


### Lets see what we achieved

In [7]:
from time import sleep
from IPython.display import clear_output  # <<<--- Used for a video-like experience by viewing frame by frame

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['rewards']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 687
State: 0
Action: 5
Reward: 20


# Inference

Not good. Our agent takes thousands of timesteps and makes lots of wrong drop offs to deliver just one passenger to the right destination.

This is because we aren't learning from past experience. We can run this over and over, and it will never optimize. The agent has no memory of which action was best for each state, which is exactly what Reinforcement Learning will do for us.


# Reinforcement Learning with Q Table

Q(state,action) ← (1−α) * Q(state,action) + α(reward + γ * maxaQ(next state,all actions))



Where:

- α (alpha) is the learning rate (0<α≤1) - Just like in supervised learning settings, α is the extent to which our Q-values are being updated in every iteration.

- γ (gamma) is the discount factor (0≤γ≤1) - determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas, a discount factor of 0 makes our agent consider only immediate reward, hence making it greedy.




In [8]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [9]:
q_table.shape

(500, 6)

In [10]:
np.random.uniform(0, 1)

0.29561120358809445

There's a tradeoff between exploration (choosing a random action) and exploitation (choosing actions based on already learned Q-values). We want to prevent the action from always taking the same route, and possibly overfitting, so we'll be introducing another parameter called ϵ "epsilon" to cater to this during training.

Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further. Lower epsilon value results in episodes with more penalties (on average) which is obvious because we are exploring and making random decisions.

In [11]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 25.5 s


In [12]:
q_table[328]


array([ -2.41328485,  -2.27325184,  -2.40657681,  -2.36074462,
       -11.07112827, -10.99978058])

In [13]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.49
Average penalties per episode: 0.0


In [14]:
# state = env.encode(3, 1, 1, 0) # (taxi row, taxi column, passenger index, destination index)
# print("State:", state)

# env.s = state
state = env.reset()
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [15]:
steps, penalties, reward = 0, 0, 0
done = False
sleep(1)
print("####")
while not done:
    env.render()
    sleep(0.5)
    clear_output(wait=True)
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1

    steps += 1
print(f"{steps} taken for Completion")

15 taken for Completion
